<h1 align="center"><font color="yellow">LangChain: Chains</font></h1>

<font color="yellow">Data Scientist.: Dr.Eddy Giusepe Chirinos Isidro</font>

Estudaremos:

* LLMChain

* Sequential Chains
  
  * SimpleSequentialChain
  * SequentialChain

* Router Chain

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file


In [3]:
import pandas as pd

df = pd.read_csv('./Q&A_operadora_vivo.csv')
df.head()


# NOTA: No DataFrame original têm as colunas: "Product" e "Review" (a qual é a abordagem de satisfação do cliente)

,question,answer
0,❗ Como fazer uma reclamação da Vivo?,"Para reclamar com a Vivo, basta ligar no 103 15"
1,🏠 Como sei que a cobertura da Vivo Fibra atend...,A Vivo está presente em mais de 5.000 estados ...
2,📞 Qual o telefone de SAC da Vivo Internet?,"Para atendimento, problemas de conexão ou cont..."
3,📲 Como entrar em contato com a Vivo para emiss...,"Para emitir a segunda via da sua fatura, basta..."
4,📳 Qual o telefone da Vivo para falar com atend...,O número de telefone da Vivo para falar com at...


In [4]:
df.shape

(26, 2)

# <font color="red">LLMChain</font>

In [5]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain


llm = ChatOpenAI(temperature=0.0)

In [6]:

prompt = ChatPromptTemplate.from_template(
    " Qual é o melhor nome para descrever \
    uma empresa que fabrica {product}?"
)

In [7]:
chain = LLMChain(llm=llm, prompt=prompt)

In [8]:
product = "Persianas de diferentes cores e impermeáveis."

chain.run(product)


'Um possível nome para descrever uma empresa que fabrica persianas de diferentes cores e impermeáveis poderia ser "ColorBlind Shades" (Persianas Coloridas e Impermeáveis).'

# <font color="red">SimpleSequentialChain</font>

In [9]:
from langchain.chains import SimpleSequentialChain


llm = ChatOpenAI(temperature=0.0)


# Prompt template 1:
first_prompt = ChatPromptTemplate.from_template(
    "Qual é o melhor nome para descrever \
    uma empresa que fabrica {product}?"
)


# Chain 1:
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [10]:
# Prompt template 2:
second_prompt = ChatPromptTemplate.from_template(
    "Escreva uma descrição de 20 palavras para a \
    seguinte empresa:{company_name}"
)


# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [11]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [12]:
overall_simple_chain.run(product)




> Entering new  chain...
Um possível nome para descrever uma empresa que fabrica persianas de diferentes cores e impermeáveis poderia ser "ColorBlind" ou "AquaShade".
"ColorBlind" é uma empresa especializada na fabricação de persianas impermeáveis em uma ampla variedade de cores vibrantes.

> Finished chain.


'"ColorBlind" é uma empresa especializada na fabricação de persianas impermeáveis em uma ampla variedade de cores vibrantes.'

# <font color="red">SequentialChain</font>

In [13]:
from langchain.chains import SequentialChain


llm = ChatOpenAI(temperature=0.0)

# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Traduza a seguinte resenha para o Inglês:"
    "\n\n{Review}"
)
# chain 1: input = Review  and  output = English_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="English_Review"
                    )


In [14]:
second_prompt = ChatPromptTemplate.from_template(
    "Você pode resumir a seguinte resenha em 1 frase:"
    "\n\n{English_Review}"
)
# chain 2: input = English_Review   and   output = summary
chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summary"
                    )


In [15]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "Em que idioma está a seguinte resenha:\n\n{Review}"
)

# chain 3: input = Review   and  output = language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                      )

In [16]:
# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Escreva uma resposta de acompanhamento para o seguinte"
    "resumo no idioma especificado:"
    "\n\nResumo: {summary}\n\nIdioma: {language}"
)

# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )


In [21]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary", "language", "followup_message"],
    verbose=True
)


In [22]:
review = df.answer[0]
overall_chain(review)




> Entering new  chain...

> Finished chain.


{'Review': 'Para reclamar com a Vivo, basta ligar no 103 15',
 'English_Review': 'To complain to Vivo, just call 103 15.',
 'summary': 'To file a complaint with Vivo, dial 103 15.',
 'language': 'A resenha está em português.',
 'followup_message': 'Caro cliente,\n\nAgradecemos por entrar em contato conosco e por compartilhar seu resumo em português. Se você está procurando registrar uma reclamação com a Vivo, sugerimos que você disque o número 103 15. Nossa equipe de atendimento ao cliente estará pronta para ajudá-lo e resolver qualquer problema que você possa ter.\n\nLembramos que estamos disponíveis para atendê-lo 24 horas por dia, 7 dias por semana. Se você tiver alguma dúvida adicional ou precisar de assistência adicional, não hesite em nos contatar novamente.\n\nAgradecemos por escolher a Vivo como seu provedor de serviços e esperamos poder atendê-lo da melhor maneira possível.\n\nAtenciosamente,\nEquipe de Atendimento ao Cliente da Vivo'}

# <font color="red">Router Chain</font>

In [23]:
physics_template = """Você é um professor de física muito inteligente. \
Você é ótimo em responder perguntas sobre física de maneira concisa e fácil \
de entender. Quando você não sabe a resposta para uma pergunta, você admite \
que não sabe. \

Aqui está uma pergunta: {input}"""


math_template = """Você é um matemático muito bom. Você é ótimo em responder \
perguntas de matemática. Você é tão bom porque é capaz de dividir problemas difíceis \
em suas partes componentes, responder às partes componentes e depois reuni-las para \
responder à pergunta mais ampla. \

Aqui está uma pergunta: {input}"""

history_template = """Você é um historiador muito bom. Você tem um excelente conhecimento \
e compreensão de pessoas, eventos e contextos de uma variedade de períodos históricos. \
Você tem a capacidade de pensar, refletir, debater, discutir e avaliar o passado. Você tem \
respeito pelas evidências históricas e a capacidade de usá-las para apoiar suas explicações \
e julgamentos. \

Aqui está uma pergunta:
{input}"""


computerscience_template = """Você é um cientista da computação de sucesso. Você tem paixão \
por criatividade, colaboração, visão de futuro, confiança, fortes capacidades de resolução de \
problemas, compreensão de teorias e algoritmos e excelentes habilidades de comunicação. \
Você é ótimo em responder perguntas de codificação. Você é tão bom porque sabe como resolver \
um problema descrevendo a solução em etapas imperativas que uma máquina pode interpretar \
facilmente e sabe como escolher uma solução que tenha um bom equilíbrio entre complexidade de \
tempo e complexidade de espaço. \

Aqui está uma pergunta:
{input}"""



In [24]:
prompt_infos = [
    {
        "name": "Física", 
        "description": "Bom para responder perguntas sobre física", 
        "prompt_template": physics_template
    },
    {
        "name": "Matemática", 
        "description": "Bom para responder questões de matemática", 
        "prompt_template": math_template
    },
    {
        "name": "História", 
        "description": "Bom para responder perguntas de história", 
        "prompt_template": history_template
    },
    {
        "name": "Ciência da Computação", 
        "description": "Bom para responder perguntas de ciência da computação", 
        "prompt_template": computerscience_template
    }
]

In [25]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate


llm = ChatOpenAI(temperature=0)

In [26]:
destination_chains = {}

for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)

    chain = LLMChain(llm=llm, prompt=prompt)
    
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)


In [27]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)


In [28]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Dada uma entrada de texto bruto para um modelo de idioma, \
selecione o prompt do modelo mais adequado para a entrada. Você receberá os nomes dos prompts \
disponíveis e uma descrição do que o prompt é mais adequado. Você também pode revisar a entrada \
original se achar que a revisão levará a uma melhor resposta do modelo de linguagem.

<< FORMATTING >>
Retorne um snippet de código markdown com um objeto JSON formatado para se parecer com:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [29]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)


In [30]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )


In [31]:
chain.run("O que é radiação de corpo negro?")




> Entering new  chain...
Física: {'input': 'O que é radiação de corpo negro?'}
> Finished chain.


'A radiação de corpo negro é um conceito importante na física que descreve a radiação eletromagnética emitida por um objeto que absorve toda a radiação incidente sobre ele, sem refletir ou transmitir qualquer parte dela. Um corpo negro ideal é um objeto hipotético que absorve toda a radiação que incide sobre ele, independentemente da frequência ou comprimento de onda. \n\nA radiação de corpo negro é caracterizada por sua distribuição espectral de energia, conhecida como distribuição de Planck. Essa distribuição descreve como a energia é distribuída em diferentes comprimentos de onda ou frequências. A distribuição de Planck mostra que a radiação de corpo negro tem um pico de intensidade em uma determinada frequência, que está relacionada à temperatura do objeto. \n\nA radiação de corpo negro é um fenômeno importante em várias áreas da física, como a termodinâmica e a teoria quântica. Ela desempenhou um papel fundamental no desenvolvimento da teoria quântica, pois a distribuição de Planc

In [32]:
chain.run("Quanto é 2 + 2")



> Entering new  chain...
Matemática: {'input': 'Quanto é 2 + 2'}
> Finished chain.


'A resposta para a pergunta "Quanto é 2 + 2?" é 4.'

In [33]:
chain.run("Por que todas as células do nosso corpo contêm DNA?")



> Entering new  chain...
Ciência da Computação: {'input': 'Por que todas as células do nosso corpo contêm DNA?'}
> Finished chain.


'Todas as células do nosso corpo contêm DNA porque o DNA é a molécula responsável por armazenar e transmitir informações genéticas. O DNA contém instruções para a síntese de proteínas, que são essenciais para o funcionamento e desenvolvimento de um organismo.\n\nCada célula do nosso corpo possui um conjunto completo de DNA, que contém todas as informações necessárias para a formação e manutenção do organismo. Essas informações são transmitidas de geração em geração e são responsáveis por determinar as características físicas e biológicas de um indivíduo.\n\nAlém disso, o DNA também desempenha um papel fundamental na regulação dos processos celulares. Ele controla a expressão dos genes, ou seja, determina quais genes serão ativados ou desativados em cada tipo de célula. Isso permite que as células desempenhem funções específicas de acordo com sua especialização, como as células musculares, células nervosas, células sanguíneas, entre outras.\n\nEm resumo, todas as células do nosso corpo 